<h1>Extracting and Visualizing Stock Data</h1>



<h2>Description</h2>

Extracting essential data from a dataset and displaying it is a necessary part of data science allowing stakeholders to make correct decisions based on the data. In this notebook, I will extract some stock data and then display this data in a graph. In this notebook, I will extract some stock data and then display this data in a graph.


<h2>Objective</h2>

My goal is to extract financial data like historical share price and querterly revenue reportings from various sources using python libraries and webscraping on popular stocks. After collecting this data I will visualize it in a dashboard to identify patterns or trends. Stocks I will work with are Tesla and GameStop.

In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function

In this section, I define the function `make_graph`.


In [4]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance to Extract Stock Data

Using the `Ticker` function, the ticker symbol of the stock I want to extract data is entered to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [5]:
Tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history`, I extract stock information and save it in a dataframe named `tesla_data`.  The `period` parameter is set to `max` so I get information for the maximum amount of time.


In [7]:
tesla_data = Tesla.history(period="max")
df=tesla_data

I then reset the index, save, and display the first five rows of the `tesla_data` dataframe using the `head` function. 

In [8]:
df.reset_index(inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data


I use the `requests` library to download the webpage [https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue](https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Then save the text of the response as a variable named `html_data`.


In [10]:
url= "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

I Parse the html data using `beautiful_soup`. Then scrape the table on the webpage into a dataframe, it should look like the table from the website.


In [12]:
soup = BeautifulSoup(html_data,"html.parser")

I use beautiful soup extract the table with Tesla Quarterly Revenue and store it into a dataframe named `tesla_revenue`. The comma and dollar sign is removed from the `Revenue` column.


In [13]:
df=pd.read_html(html_data,header=0)
table=soup.find_all('table')
second_table= table[1]
tesla_revenue= pd.DataFrame(columns=["Date","Revenue"])
for row in second_table.find("tbody").find_all("tr"):
    col= row.find_all('td')
    date= col[0].string
    revenue= col[1].string
    tesla_revenue= tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index = True)
tesla_revenue["Revenue"]= tesla_revenue['Revenue'].str.replace('$','').str.replace(',','')
print(tesla_revenue)

C:\Users\ana02\AppData\Local\Temp\ipykernel_11788\3212783899.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(html_data,header=0)


ValueError: No tables found

I remove the columns in the dataframe that are empty strings


In [14]:
tesla_revenue.dropna(subset=['Revenue'], inplace=True)
for i in tesla_revenue : tesla_revenue[i] = tesla_revenue[i].astype(str)
#tesla_revenue['Date'] = tesla_revenue['Date'].astype(str)
#tesla_revenue['Revenue'] = tesla_revenue['Revenue'].astype(str)

NameError: name 'tesla_revenue' is not defined

I display the last 5 row of the `tesla_revenue` dataframe using the `tail` function. 

In [15]:
tesla_revenue.tail(5)

NameError: name 'tesla_revenue' is not defined

## Using yfinance to Extract Stock Data pt.2

Using the `Ticker` function, I enter the ticker symbol of the stock I want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [18]:
GME= yf.Ticker("GME")

Using the ticker object and the function `history`, I extract stock information and save it in a dataframe named `gme_data`. I then set the `period` parameter to `max` so I get information for the maximum amount of time.


In [19]:
GME_share_price_data = GME.history(period="max")
df=GME_share_price_data

I reset the index, save, and display the first five rows of the `gme_data` dataframe using the `head` function. 


In [20]:
df.reset_index(inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Using Webscraping to Extract GME Revenue Data

I use the `requests` library to download the webpage [https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue](https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Then I save the text of the response as a variable named `html_data`.


In [21]:
url= "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text

I parse the html data using `beautiful_soup`and scrape the table on the webpage into a dataframe. It should look like the table from the website.


In [22]:
soup = BeautifulSoup(html_data,"html.parser")

Using beautiful soup, I extract the table with Tesla Quarterly Revenue and store it into a dataframe named `gme_revenue`.  I make sure the comma and dollar sign is removed from the `Revenue` column.


In [23]:
df=pd.read_html(html_data,header=0)
table=soup.find_all('table')
second_table= table[1]
gme_revenue= pd.DataFrame(columns=["Date","Revenue"])
for row in second_table.find("tbody").find_all("tr"):
    col= row.find_all('td')
    date= col[0].string
    revenue= col[1].string
    gme_revenue= gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index = True)
gme_revenue["Revenue"]= gme_revenue['Revenue'].str.replace('$','').str.replace(',','')

gme_revenue.dropna(subset=['Revenue'], inplace=True)
for i in gme_revenue : gme_revenue[i] = gme_revenue[i].astype(str)
print(gme_revenue)

C:\Users\ana02\AppData\Local\Temp\ipykernel_11788\3174127543.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(html_data,header=0)


ValueError: No tables found

I display the last five rows of the `gme_revenue` dataframe using the `tail` function. 

In [24]:
gme_revenue.tail()

NameError: name 'gme_revenue' is not defined

## Plotting Tesla Stock Graph

I use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph.


In [26]:
make_graph(tesla_data,tesla_revenue,"Tesla Stock Data")


NameError: name 'tesla_revenue' is not defined

## Plotting GameStop Stock Graph

I use the `make_graph` function to graph the GameStop Stock Data and give the graph a title.


In [27]:
make_graph(GME_share_price_data,gme_revenue,"GameStop Stock Data")


NameError: name 'gme_revenue' is not defined